In [1]:
import numpy as np
from numpy import nan

from verbose import print_state, print_experience, print_buffer, print_shapes, action_available, print_interaction, print_cummulative_reward
# from utils import 
from constant import *
from plot import plot_evolution


from DQN import DQNAgent
from env import Env
from utils_class import EpsilonGreedy


%load_ext autoreload
%autoreload 2


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
sess = InteractiveSession(config=config)


2022-06-08 07:04:02.533357: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-06-08 07:04:02.533386: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-06-08 07:04:04.171559: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-08 07:04:04.197009: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-08 07:04:04.197325: W tensorflow/s

In [3]:
STACK_SIZE = 3
NBR_STACK = 2

env = Env(STACK_SIZE)
strategy = EpsilonGreedy(MIN_EPSILON, START_EPSILON, DECAY)
agent = DQNAgent(NBR_STACK, STACK_SIZE, len(env.action_space()))

In [4]:
print_state(env, shape=True)

---State t---
-A-	-B-
___________	
|53.0	nan|
|1.0	nan|
|28.0	nan|
___________	
A shape : (3,)
B shape : (3,)
___________	
------------
Env : in progress
------------


In [97]:
episode_durations = list()
cummulative_reward = list()
epsilon = list()

for episode in range(EPISODE):
    
    print(f"Episode : [{episode}]")
    env.reset(STACK_SIZE)
    
    state = env.get_state()
    print_state(env)
    
    timesteps = 0
    while (1):
        
        # action_available(env)
        # print_state(env)
        timesteps += 1
        
        exploration_rate = strategy.get_exploration_rate(timesteps)
        
        action = env.choose_action(state, agent.policy_model, exploration_rate)
        reward = env.reward()
        next_state = env.get_state()
        print_interaction(exploration_rate, action, reward)
        
        
        
        experience = env.create_experience(state, action, next_state, reward)
        env.replaymemory.push(experience)
        state = next_state
        
        # print_experience(experience)
        if env.replaymemory.can_provide_sample(BATCH_SIZE):
            print("======== START TRAIN ========")
            
            buffer = env.replaymemory.get_sample(BATCH_SIZE)
            # print_buffer(buffer)
            
            states, actions, next_states, rewards, dones = env.replaymemory.extract_value(buffer)
            # print_shapes(states, next_states)
            agent.train(states, next_states, buffer)
        
        # print(f"Env : {env.state()}")
        if (env.state() == 'done'):
            print_state(env)
            print_cummulative_reward(env)
            epsilon.append(strategy.epsilon)
            episode_durations.append(timesteps)
            cummulative_reward.append(env.cummulative_reward)
            
            plot_evolution(episode, episode_durations, cummulative_reward)
            break
            
    
    

Episode : [0]
---State t---
-A-	-B-
___________	
|22.0	nan|
|96.0	nan|
|53.0	nan|
___________	
------------
Env : in progress
------------
Exploration
----------------------------------
Action [swap_a]
Reward [-1]
Epsilon [0.9991504248583688]
----------------------------------
Exploration
----------------------------------
Action [rotate_a]
Reward [-1]
Epsilon [0.9983016988672331]
----------------------------------
Exploration
----------------------------------
Action [inverse_rotate_a]
Reward [-6]
Epsilon [0.9974538211778671]
----------------------------------
Exploration
----------------------------------
Action [push_b]
Reward [-1]
Epsilon [0.9966067909423928]
----------------------------------
Exploration
----------------------------------
Action [push_b]
Reward [-1]
Epsilon [0.99576060731378]
----------------------------------
Exploration
----------------------------------
Action [inverse_rotate_a]
Reward [-5]
Epsilon [0.994915269445845]
----------------------------------
Explorat

In [83]:
env.replaymemory.print_history(BATCH_SIZE)

Actions : 7 	 Reward : 99
